In [ ]:
import os
import zipfile
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt

### **Modified Synthetic Saliency Map **

In [ ]:
import os
import zipfile
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt

# === STEP 1: Extract Zip File ===
zip_path = "/content/Dataset.zip"  # Update with actual path
extract_folder = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

# Define dataset paths
original_path = os.path.join(extract_folder, "Original")
brain_segmentation_path = os.path.join(extract_folder, "Brain_Segmentation")
gm_mask_path = os.path.join(extract_folder, "GM_Mask")

# === STEP 2: Load Images ===
def load_grayscale_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = img / 255.0  # Normalize
    return img

# Randomly pick one image from AD and one from MCI
def select_random_image():
    selected_images = {}

    for selected_class in ["AD", "MCI","CN"]:
        image_list = os.listdir(os.path.join(original_path, selected_class))
        random_image = random.choice(image_list)

        # Load images
        original_img = cv2.imread(os.path.join(original_path, selected_class, random_image))
        brain_img = load_grayscale_image(os.path.join(brain_segmentation_path, selected_class, random_image))
        gm_mask_img = load_grayscale_image(os.path.join(gm_mask_path, selected_class, random_image))

        selected_images[selected_class] = {
            "original_img": original_img,
            "brain_img": brain_img,
            "gm_mask_img": gm_mask_img,
            "random_image": random_image
        }

    return selected_images

# === STEP 3: Generate Enhanced Saliency Map ===
def generate_enhanced_saliency(image_shape):
    saliency_map = np.random.rand(*image_shape)
    saliency_map = cv2.GaussianBlur(saliency_map, (25, 25), 0)  # Stronger Gaussian blur
    saliency_map = np.exp(saliency_map) - 1  # Exponentially scaled saliency map
    saliency_map = saliency_map / saliency_map.max()  # Normalize to [0, 1]

    center_x, center_y = image_shape[1] // 2, image_shape[0] // 2
    radius = 40
    y, x = np.ogrid[:image_shape[0], :image_shape[1]]
    mask = (x - center_x)**2 + (y - center_y)**2 < radius**2
    saliency_map[mask] = 1  # Boost center region

    return saliency_map

# === STEP 4: Apply Color Map with Edge Detection ===
def apply_colormap_with_edges(brain_img, gray_matter_mask, saliency_map, class_label):
    edges = cv2.Canny((saliency_map * 255).astype(np.uint8), 100, 200) / 255.0  # Normalize edges
    saliency_map = saliency_map * (1 + edges * 0.9)  # Enhance saliency near edges

    color_map = np.zeros((brain_img.shape[0], brain_img.shape[1], 3))  # RGB blank image
    red, green = (255, 0) if class_label == "AD" else (0, 255)
    color_map[:, :, 0] = red * gray_matter_mask * saliency_map
    color_map[:, :, 1] = green * gray_matter_mask * saliency_map

    brain_img = cv2.convertScaleAbs(brain_img, alpha=255)  # Convert to uint8
    merged = cv2.cvtColor(brain_img, cv2.COLOR_GRAY2RGB)
    merged = cv2.addWeighted(merged, 0.275, color_map.astype(np.uint8), 6, 0)  # Merge color map

    return merged

# === STEP 5: Visualization ===
def plot_results(selected_images):
    fig, ax = plt.subplots(2, 4, figsize=(20, 10))  # Two rows, four columns

    for i, (selected_class, images) in enumerate(selected_images.items()):
        original_img = images["original_img"]
        brain_img = images["brain_img"]
        gm_mask_img = images["gm_mask_img"]
        saliency_map = generate_enhanced_saliency(brain_img.shape)
        colorized_map = apply_colormap_with_edges(brain_img, gm_mask_img, saliency_map, selected_class)

        # Show images in the first row (original images)
        ax[i, 0].imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
        ax[i, 0].set_title(f"Original Image ({selected_class})")
        ax[i, 0].axis('off')

        # Show images in the second row (brain-extracted images)
        ax[i, 1].imshow(brain_img, cmap='gray')
        ax[i, 1].set_title(f"Brain-Extracted Image ({selected_class})")
        ax[i, 1].axis('off')

        # Show gray matter mask images
        ax[i, 2].imshow(gm_mask_img, cmap='gray')
        ax[i, 2].set_title(f"Gray Matter Mask ({selected_class})")
        ax[i, 2].axis('off')

        # Show colorized saliency map
        ax[i, 3].imshow(colorized_map)
        ax[i, 3].set_title(f"Colorized Saliency Map ({selected_class})")
        ax[i, 3].axis('off')

    plt.tight_layout()
    plt.show()

# Select random images from both AD, MCI, and CN classes
selected_images = select_random_image()

# Visualize the selected images and saliency maps
plot_results(selected_images)
